Molecular Dynamics
==================

Thus far we have studied the dynamics behavior of systems with only a
few particles. However, many systems in nature such as gases, liquids
and solids contain many mutually interacting particles. A typical
problem involves $%
10^{23}-10^{25}$ molecules which, to a good approximation, obey the laws
of classical physics. Although the intermolecular forces produce a
complicated trajectory for each particle, the observable properties of
the whole are averages that do not depend on the individual behavior.
The motion is completely determined by the classical equations of
motion, and all the physics is contained in the intramolecular
potential, *i.e.* the interactions.

The problem consists in understanding the properties of the whole,
starting from the known interactions. The most direct approach is to
simulate the problem with a computer. The problem of simulating a system
with $%
10^{23}-10^{25}$ particles requires the power of a supercomputer, and
using parallelization techniques. However we can learn the principles
using a fraction of that number, say $10^{4}$ particles, that is
tractable in a PC. Not only we can learn about about many of the instead
of statistical mechanics, but also the basic simulation techniques. The
technique that we will learn in this section does not differ in essence
from that used in supercomputers, and it is called “molecular dynamics”.

The intermolecular potential
----------------------------


Our first goal is to identify the model we want to simulate. The first
simplification is to consider the molecules spherical, chemically inert,
and that they move according to the laws of classical mechanics. We also
assume that the force between molecules depends only on the distance
between them. The form of this potential for electrically neutral atoms
can be constructed from a detailed first principles quantum mechanical
calculation. Such calculation can be complicated, so we will consider a
phenomenological form. The most important features are: **strong
repulsion at short distances, weak attraction at large separations**.
The repulsion originates from the electrostatic interaction between
equally charged particles, and the long range attraction from the mutual
polarization of the electronic clouds, known as “van de Waals” force.

Below we list some model potentials:

$$\begin{array}{ll}
& \mathrm{Hard\,\, Spheres} & V(r)=\infty    \mathrm{\,\,if\,\,} r<r_{o} & \mathrm{First\,\, approximation} \\    
&  &    V(r)=0  \mathrm{\,\,if\,\,} r\geq r_{o}  & \mathrm{in\,\, many\,\, applications} \\
& & & \\
& \mathrm{Lennard-Jones} & V(r)=4\epsilon \left[ \left( \frac{r}{\sigma }\right)^{-12}-\left( \frac{r}{\sigma}\right)^{-6}\right] &  \mathrm{Noble\,\, gas\,\, atoms;\,\, nearly\,\, spherical\,\, molecules} \\
& & & \\
& \mathrm{Harmonic} & V(r)=A(r-r_{o})^{2} & \mathrm{Intramolecular\,\, bonds}
\end{array}$$

Tricks of the trade
-------------------

### Units

Meters and kilograms are not meant to measure molecules. Using these
units would imply the clumsy manipulation of small numbers. We want to
choose units of mass, energy, and length such that the values of the
quantities are always of the order $1$. For instance, in the case of the
Lennard-Jones potential is wise to use $\sigma $ as a unit of length,
$\epsilon $ as a unit of energy, and $m_{0}=1AMU$ as the unit of mass.
Re-scaling by these quantities, the potential would be
$$V^{*}(r)=4\left[ u^{-12}-u^{-6}\right] ,$$ where $V^{*}=V/\epsilon $
and $u=r/\sigma $. The advantages are that the Lennard-Jones parameters
never show in the formulas, and that we are spared of many
computationally expensive calculations. With this choice of units, we
find that time is measured in units of $t_{0}=\sqrt{m_{0}\sigma
^{2}/\epsilon }$. For the electrical charge, the natural unit is the
charge of the electron $e$, and for temperature, $\epsilon /k$. After
finishing the simulation, we transform the results back to the original
units to be compared with the experimental data.

<!--
#### Challenge

Figure out how to rescale the units for argon atoms interacting via a Lennard-Jones potential.
The parameters for Ar are $\epsilon = 0.0104 eV$, $\sigma = 3.40$ ${\buildrel _{\circ} \over {\mathrm{A}}}$. The mass of one argon atom is $m = 39.948 AMU$.
!-->

<!--
**Hint:** The program should use eVs as the unit of energy and temperature (so Boltzmann’s constant is $k_B = 1$), and ${\buildrel _{\circ} \over {\mathrm{A}}}$ for the unit of distance. Time is measured in femtoseconds.
Since the energy is in eV as a function of ${\buildrel_{\circ} \over {\mathrm{A}}}$, the force will be in units of $eV/{\buildrel_{\circ} \over {\mathrm{A}}}$. 
!-->

### Boundary conditions

![pbc](pbc.png)
#### Example of periodic boundary conditions in 2D. Note that particle 1 is about to leave the left face of the central cell and to enter the center cell trhough the right face. The minimum image distance convention inplies that the separation between particles 1 and 2 is given by the bold line

<!--
Since the size of our system is typically 10-100 molecular diameters,
this is certainly not a good representation for a macroscopic sample
because most of the particles will be situated near a “wall” or
“boundary”.!-->

To minimize the effects of the boundaries and to simulate
more closely the properties of the macroscopic system, it is convenient
to choose “periodic boundary conditions”. This means that our basic box
containing $N$ particles is surrounded by images (or replicas) of
itself. This is equivalent to wrap the coordinates around the
boundaries: when one particles hits a wall, instead of bouncing pack, it
reappears on the other side of the box. This means that the system has
the topology of a torus. If the linear size of the box is $L$, the
maximum separation between particles is $L/2$.

 In this way the number of
particles in a given box is always conserved. A particle crossing the
right boundary is automatically replaced by a particle entering the
left, and vice versa (Adding $2L$ before the modulo operation is to
catch any runaway particles with $x_{i}<-L$).

To compute the potential energy or the force between two particles $i$
and $j
$ one augments the periodic boundary conditions we have to adopt the
so-called “nearest image convention”. Imagine that the two particles on
opposite sides of the box. The convention dictates that we have to adopt
the minimum distance across the walls, or between images in the
neighboring replicas. If $\Delta x_{ij}=x_{j}-x_{i}$ is larger than
$L/2$, then the particle $j$ will be disregarded as an interaction
partner of $i$, with its left image, having coordinate $x_{j}-L$ in its
place. In practice this means simply that when calculating $V(r_{ij})$
we have to use the quantity $\Delta
x_{ij}-L$ instead of $\Delta x_{ij}$. An analogous rule holds for
$\Delta
x_{ij}\leq -L/2$ and for the other coordinates.

The rule can be expressed by
$$\Delta x=\Delta x-L\mathrm{int}\left( \frac{\Delta x}{L}\right).$$

The rules  can be replaced by "if"
expressions. In a parallel code this is counter productive, but in a
serial code may be preferable. The optimal choice should be based on
benchmarks.

### Starting configuration

![initial](initial.png)
#### Initial configuration with particles in vertices of a a) triangular; b) rectangular lattice

Picking the right starting configuration is
not trivial. The first choice would be to place the molecules randomly
distributed, but this would give rise to large starting energies and
forces, since many pairs would be placed at unphysical short distances.
It is therefore customary to place the particles in the vertices of a
some crystal lattice (face centered cubic –or triangular–, for
instance). The $x$ and $y$ components of the velocities can be picked
randomly in an interval $[-v_{\max },v_{\max }]$. Before starting the
proper simulation and the measurement of the physical quantities, it is
necessary to perform a “thermalization” run to let the system relax to a
situation of dynamical and thermal equilibrium.

Other possibilities consist of arranging the particles in a regular periodic structure as shown in the figure. The velocities are again picked randomly, and the system must evolve for some time to thermalize.

### Adjusting density and temperature

Since the number of particles is usually fixed and conserved, the way to
achieve the desired density is to adjust the volume. This is done
scaling all coordinates by a suitable factor.

In Molecular Dynamics, the temperature is a measurable quantity. Kinetic
theory tells us that $$T=m\langle |\mathbf{v}|^{2}\rangle /3k,$$ where
$k$ is the Boltzmann’s constant, and $\langle |\mathbf{v}|^{2}\rangle $
is the averaged square of the velocity. In a simulation, we have to
average this quantity over a number of MD steps to get a measurement of
$T$. If we want to simulate a particular temperature, we rescale each
component of the velocity vector of every particle by
$\sqrt{T_{desired}/T_{actual}}$. Obviously $T$ is a fluctuating quantity
and can therefore be adjusted only approximately.


MD simulation: Continuous (Analytic) potentials
------------------------------------

The interaction between hard particles was considered as an
instantaneous collision process, with forces of infinite strength acting
instantaneously during infinitely short times. A dynamical equation is
of no use in such model, and it was therefore appropriate to calculate
the collision laws to obtain the new velocities. In between collisions,
the particles fly freely without acceleration. In contrast, for a
continuous varying potential, the particles are either in a force field,
or interact with some other particle at long distances and at any time.
Therefore
$$\mathbf{\ddot{r}}=\frac{1}{m}\mathbf{F}_{i}=\frac{1}{m}\sum_{j\neq i}\mathbf{%
f}_{ij}(t)$$ with $$\mathbf{f}_{ij}=-\nabla _{i}V(r_{ij})$$ where $V$ is
the interaction potential (*e.g.* Lennard-Jones).

When evaluating the total force acting on a particle we apply periodic
boundary conditions and the nearest image convention. In this way we may
determine the sum of all the forces acting on a particle. A popular
method to integrate the equations of motion is the Verlet algorithm that we learned in the previous 
sections.
<!---:
$$\mathbf{r}_{i}(t_{n+1})=2\mathbf{r}_{i}(t_{n})-\mathbf{r}_{i}(t_{n-1})+%
\mathbf{F}_{i}(t_{n})(\Delta t)^{2}$$ (Notice that this is not a self
starting algorithm)
!-->

### Exercise 4.1: Approach to equilibrium I 

Write the code for simulating a one-dimensional system of particles
interacting via a Lennard-Jones potential. Consider $N=12$ equally-spaced particles on a line of length 25. Employ periodic boundary conditions. Suppose that the particles are initially
constrained to be in the left part of the box and they are equally spaced. At $t=0$ the constraint is removed and the particles
move freely throughout the entire box. To initialize your system, use $v_{\max }=1.0$ (the maximum initial speed) and a timestep $\Delta t=0.02$.

1.  Observe a sufficient number of snapshots for the particles to move significantly from their original positions. Does the system appear to become more or less random?

2.  From the visual snapshots of the trajectories, estimate the time for
    the system to reach equilibrium. What is your qualitative criterion
    for equilibrium?

3.  Now repeat the simulation with an maximum initial velocity of 0.1. Are there qualitative differences in the dynamics? If so, why?

### Exercise 4.2: Approach to equilibrium II

For the above system, a more rigorous method for describing equilibration is to show that ensemble averages converge. For both systems, calculate the probability distribution of particle positions. Repeat the calculation over a simulation of 1000, 2000.... 10000 timesteps. That is, the probability is taken over all particles, and all times. Based on this, does it appear your simulations are converging, in terms of their statistical properties?

### Exercise 4.3: The flying ice cube

Repeat the calculation above, but initialize all particles with the same velocity and spaced at $r_{min}$.  Calculate the probability distribution as a function of time (intervals of 10000 steps). Once the distributions have converged, does your simulation represent a statistical mechanical ensemble?
